<h1>Point_E Model Scanning Sample</h1>

<h3>Point_E Image To PointCloud<h3/>

In [2]:
import tqdm
from AIScan import CloudSampler
from AIScan.Models import CreateModel
from Utils.Data import getPathImages, getPathNPBinaries
from Utils.Torch import UseBestTorchDevice

from point_e.util.point_cloud import PointCloud
from point_e.util.mesh import TriMesh
from point_e.util.pc_to_mesh import marching_cubes_mesh
from point_e.util.plotting import plot_point_cloud

srcPath = './data/Skull'

In [5]:
# Loading Sampler For Chosen Cloud

sampler = CloudSampler.CreateCloudSampler('base1B')

Using cuda device for Torch Model.
Loading base1B Model


c:\Users\finnd\AppData\Local\Programs\Python\Python310\lib\site-packages\clip\clip.py:57: UserWarning: d:\Programming\Projects\Dissertation-2022_2023\Dissertation\samples\point_e_model_cache\ViT-L-14.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|███████████████████████████████████████| 890M/890M [02:51<00:00, 5.45MiB/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 6.00 GiB total capacity; 5.32 GiB already allocated; 0 bytes free; 5.33 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Loading Images From Provided Source Path (Provided In First Cell)

images = getPathImages(srcPath)
if (len(images) == 0):
    print("--__ No Images Found __--")
    exit()

In [ ]:
# Generate Samples from the Sampler, Using Provided Images

samples = None

for x in tqdm.tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=images))):
    samples = x

del images

In [ ]:
# Save All Generated Samples To Cloud Files, 

figs = []

for i, cloud in enumerate(sampler.output_to_point_clouds(samples)):
    figs[i] = plot_point_cloud(cloud, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

    with open(f'{srcPath}/cloud_{i}.npz', 'wb') as f:
        cloud.save(f)

del sampler
del samples

In [ ]:
# Clear All Variables from Memory
del sampler
del samples

del figs

<h3>Point_E Pointcloud To Mesh</h3>

In [ ]:
# Create SDF Generator Model

sdfModel = CreateModel('sdf', UseBestTorchDevice())

In [ ]:
# Genreating Meshes From Clouds, Previously Saved To NPZ Files

curCloud: PointCloud = None
curMesh = None
meshes = []

for cloudPath in getPathNPBinaries(srcPath):
    curCloud = PointCloud.load(cloudPath)

    curMesh = marching_cubes_mesh(
        pc = curCloud,
        model = sdfModel,
        batch_size = 4096,
        grid_size = 128,
        progress=True
    )

    meshes.append(curMesh)